# Lab2 - PuLP Library

<b>Information on group members:</b><br>
1) 154974, Andrii Chmutov <br>
2) 158669, Vasyl Korzavatykh

In [1]:
from pulp import *
import numpy as np
import pandas as pd

## Introduction - brief tutorial on PuLP

In [2]:
# Create an LpProblem instance; LpMaximize = objective function is to be maximized
model = LpProblem(name="some-problem", sense=LpMaximize)

In [3]:
# Initialize the decision variables. We can set the name and lower bound as well.
# To create an array of variables, you can use comprehensions of LpProblem.dicts.

x1 = LpVariable(name="x1", lowBound=0)
x2 = LpVariable(name="x2", lowBound=0)

In [4]:
# An example expression
expression = 3 * x1 + 2 * x2
type(expression)

pulp.pulp.LpAffineExpression

In [5]:
# An example constraint
constraint = 2 * x1 + 3 * x2 >= 5
type(constraint)

pulp.pulp.LpConstraint

Ok, let's now use PuLP to solve the below problem:
$max$ $4x_1 + 2x_2$ <br>
s.t.<br>
     $1x_1 + 1x_2 \geq 1$ <br>
     $2x_1 + 1x_2 \leq 6$ <br>
     $-1x_1 + x_2 = 1$ <br>
     $x_1 \geq 0$ <br>
     $x_2 \geq 0$ 

In [6]:
# Problem
model = LpProblem(name="test-problem", sense=LpMaximize)

x1 = LpVariable(name="x1", lowBound=0)
x2 = LpVariable(name="x2", lowBound=0)

model += (1 * x1 + 1*x2 >= 1, "#1 constraint")
model += (2 * x1 + 1*x2 <= 6, "#2 constraint")
model += (-1 * x1 + 1*x2 == 1, "#3 constraint")

# Objective function
obj_func = 4*x1 + 2 * x2
model += obj_func

model

test-problem:
MAXIMIZE
4*x1 + 2*x2 + 0
SUBJECT TO
#1_constraint: x1 + x2 >= 1

#2_constraint: 2 x1 + x2 <= 6

#3_constraint: - x1 + x2 = 1

VARIABLES
x1 Continuous
x2 Continuous

In [7]:
# Solve the problem
status = model.solve()

In [8]:
# Print status
print(f"status: {model.status}, {LpStatus[model.status]}")

status: 1, Optimal


In [9]:
# Print objective value
print(f"objective: {model.objective.value()}")

objective: 12.000000199999999


In [10]:
# Print constraint values
for name, constraint in model.constraints.items():  print(f"{name}: {constraint.value()}")

#1_constraint: 3.3333334
#2_constraint: 9.999999983634211e-08
#3_constraint: 0.0


In [11]:
model.variables()

[x1, x2]

In [12]:
print(x1.value())
print(x2.value())

1.6666667
2.6666667


### The same code but using dictionaries and other nice tricks


In [13]:
model = LpProblem(name="some-problem", sense=LpMaximize)

In [14]:
var_names = ["First", "Second"]
x = LpVariable.dicts("x", var_names, 0)
x

{'First': x_First, 'Second': x_Second}

In [15]:
const_names = ["GE", "LE", "EQ"]
sense = [1, -1, 0] # GE, LE, EQ
coefs = [[1,1],[2,1],[-1,1]] # Matrix coefs
rhs = [1, 6, 1] 

for c, s, r, cn in zip(coefs, sense, rhs, const_names):
    expr = lpSum([x[var_names[i]] * c[i] for i in range(2)])
    model += LpConstraint(e=expr, sense = s, name = cn, rhs = r)
    
obj_coefs = [4,2]
model += lpSum([x[var_names[i]] * obj_coefs[i] for i in range(2)])
    
model

some-problem:
MAXIMIZE
4*x_First + 2*x_Second + 0
SUBJECT TO
GE: x_First + x_Second >= 1

LE: 2 x_First + x_Second <= 6

EQ: - x_First + x_Second = 1

VARIABLES
x_First Continuous
x_Second Continuous

In [16]:
status = model.solve()
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
for n in var_names: print(x[n].value())

status: 1, Optimal
objective: 12.000000199999999
1.6666667
2.6666667


# Homework - use the PuLP library to solve the following OR problem 

Johnson & Sons company manufactures 6 types of toys. Each toy is produced by utilizing at least one Machine 1-4, requiring a different production time (see Table below). For instance, product A requires 4 minutes on Machine 1, 4 minutes on Machine 2, 0 Minutes on Machine 3, and 0 minutes on Machine 4 (all machines must be utilized to produce a toy unless the production time equals 0). Each machine is available for a different number of hours per week. The company aims to identify the number (product-mix) of toys that must be manufactured to maximize the income (can be continuous). Notably, each toy can be sold for a different price. Due to market expectations, the company wants to manufacture twice as many F toys as A. Furthermore, the number of toys B should equal C. Solve this problem using the PuLP library. Prepare a report (in the jupyter notebook) containing information on the following:
<ol>
<li>The number of toys to manufacture;</li>
<li>The expected income;</li>
<li>The production time required on each machine;</li>
</ol>
Consider the total and partial values, i.e., considered for each toy A-F separately (e.g., income resulting from selling toy A). Also, answer the following questions concerning the found solution:
<ol>
<li>Which toy(s) production should be focused on?  </li>
<li>Is there a toy that can be excluded from consideration for production? </li>
<li>Is there a machine that is not fully utilized?</li>
</ol>

| Toy | Machine 1 | Machine 2 | Machine 3 | Machine 4 | Selling price |
| --- | --- | --- | --- | --- | --- |
| A | 4 (minutes!) | 4 | 0 | 0 | 2.50 USD |
| B | 0 | 3 | 3 | 0 | 1.00 USD |
| C | 5 | 0 | 2 | 5 | 4.00 USD |
| D | 2 | 4 | 0 | 4 | 3.00 USD |
| E | 0 | 4 | 5 | 2 | 3.50 USD |
| F | 2 | 2 | 1 | 1 | 4.00 USD |
| Production time limit (hours!) | 120 | 60 |  80 |  120 |  |

In [17]:
model = LpProblem(name="johnson-and-sons-problem", sense=LpMaximize)

In [18]:
var_names = ["A", "B", "C", "D", "E", "F"]
x = LpVariable.dicts("x", var_names, 0)
x

{'A': x_A, 'B': x_B, 'C': x_C, 'D': x_D, 'E': x_E, 'F': x_F}

In [19]:
const_names = ["M1", "M2", "M3", "M4", "C1", "C2"]
sense = [-1, -1, -1, -1, 0, 0]
coefs = [[4, 0, 5, 2, 0, 2], 
         [4, 3, 0, 4, 4, 2], 
         [0, 3, 2, 0, 5, 1], 
         [0, 0, 5, 4, 2, 1], 
         [-1, 0, 0, 0, 0, 2],
         [0, 1, -1, 0, 0, 0]]
rhs = [7200, 3600, 4800, 7200, 0, 0] 

for c, s, r, cn in zip(coefs, sense, rhs, const_names):
    expr = lpSum([x[var_names[i]] * c[i] for i in range(6)])
    model += LpConstraint(e=expr, sense = s, name = cn, rhs = r)
    
obj_coefs = [2.5, 1, 4, 3, 3.5, 4]
model += lpSum([x[var_names[i]] * obj_coefs[i] for i in range(6)])

model

johnson-and-sons-problem:
MAXIMIZE
2.5*x_A + 1*x_B + 4*x_C + 3*x_D + 3.5*x_E + 4*x_F + 0.0
SUBJECT TO
M1: 4 x_A + 5 x_C + 2 x_D + 2 x_F <= 7200

M2: 4 x_A + 3 x_B + 4 x_D + 4 x_E + 2 x_F <= 3600

M3: 3 x_B + 2 x_C + 5 x_E + x_F <= 4800

M4: 5 x_C + 4 x_D + 2 x_E + x_F <= 7200

C1: - x_A + 2 x_F = 0

C2: x_B - x_C = 0

VARIABLES
x_A Continuous
x_B Continuous
x_C Continuous
x_D Continuous
x_E Continuous
x_F Continuous

### 1. The number of toys to manufacture

In [20]:
status = model.solve()

n_of_toys = np.array([x[el].value() for el in x.keys()])
for n in var_names: print(f"Toy '{n}': {x[n].value()} toys")

Toy 'A': 153.19149 toys
Toy 'B': 944.68085 toys
Toy 'C': 944.68085 toys
Toy 'D': 0.0 toys
Toy 'E': 0.0 toys
Toy 'F': 76.595745 toys


### 2. The expected income

In [21]:
print(f"The expected income: {model.objective.value()}")

The expected income: 5412.765955


### 3. The production time required on each machine

In [22]:
for i in range(4):
    print(f"Machine {i+1} worked: {round(sum(n_of_toys * coefs[i]) / 60, 2)} hours")

Machine 1 worked: 91.49 hours
Machine 2 worked: 60.0 hours
Machine 3 worked: 80.0 hours
Machine 4 worked: 80.0 hours


### 4. Which toy(s) production should be focused on?

Based on the data presented below the cell, Toy C brings the most profit (approximately 3778.72$), so, logically, company should focus on its production.

In [23]:
for i, n in enumerate(var_names):
    print(f"Toy {n} brings {round(x[n].value() * obj_coefs[i], 2)}$ per week")

Toy A brings 382.98$ per week
Toy B brings 944.68$ per week
Toy C brings 3778.72$ per week
Toy D brings 0.0$ per week
Toy E brings 0.0$ per week
Toy F brings 306.38$ per week


### 5. Is there a toy that can be excluded from consideration for production?

As we've already seen above, the optimal solution implies zero production of Toys D and E, so the Toy D and the Toy E can be excluded from consideration for production. To test that we would run the same model but without Toys D and E and the constraints they provide. The numbers for produced toys should stay the same.

In [24]:
model_5 = LpProblem(name="johnson-and-sons-problem-5", sense=LpMaximize)
var_names_5 = ["A", "B", "C", "F"]
x_5 = LpVariable.dicts("x", var_names_5, 0)

const_names_5 = ["M1", "M2", "M3", "M4", "C1", "C2"]
sense_5 = [-1, -1, -1, -1, 0, 0]
coefs_5 = [[4, 0, 5, 2],
         [4, 3, 0, 2],
         [0, 3, 2, 1],
         [0, 0, 5, 1],
         [-1, 0, 0, 2],
         [0, 1, -1, 0]]
rhs_5 = [7200, 3600, 4800, 7200, 0, 0]

for c, s, r, cn in zip(coefs_5, sense_5, rhs_5, const_names_5):
    expr_5 = lpSum([x_5[var_names_5[i]] * c[i] for i in range(4)])
    model_5 += LpConstraint(e=expr_5, sense=s, name=cn, rhs=r)
    
obj_coefs_5 = [2.5, 1, 4, 4]
model_5 += lpSum([x_5[var_names_5[i]] * obj_coefs_5[i] for i in range(4)])

status = model_5.solve()

for n in var_names_5: print(f"Toy '{n}': {x_5[n].value()} toys ({x[n].value()} toys -> previously)")

Toy 'A': 153.19149 toys (153.19149 toys -> previously)
Toy 'B': 944.68085 toys (944.68085 toys -> previously)
Toy 'C': 944.68085 toys (944.68085 toys -> previously)
Toy 'F': 76.595745 toys (76.595745 toys -> previously)


As we see the number of produced toys (as expected) didn't change when Toys D and E were not considered.

### 6. Is there a machine that is not fully utilized?

As we can see from the data below, Machine 1 and Machine 4 were not used to maximal extend.

In [25]:
for i in range(4):
    print(f"Machine {i+1} is used: {round(sum(n_of_toys * coefs[i]) / 60, 2)} / {rhs[i] / 60} hours ({rhs[i] / 60 - round(sum(n_of_toys * coefs[i]) / 60, 2)} hours left)")

Machine 1 is used: 91.49 / 120.0 hours (28.510000000000005 hours left)
Machine 2 is used: 60.0 / 60.0 hours (0.0 hours left)
Machine 3 is used: 80.0 / 80.0 hours (0.0 hours left)
Machine 4 is used: 80.0 / 120.0 hours (40.0 hours left)
